In [ ]:
import io

import matplotlib.pyplot as plt
import numpy as np
import histomicstk as htk

from histomicstk.segmentation.label import trace_object_boundaries
from histomicstk.utils import merge_colinear

from skimage.data import astronaut
from skimage.measure import regionprops_table
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

import SimpleITK as sitk

from fast_slic import Slic

import time

In [ ]:
#trace and simplify superpixel boundaries
def plotit(countFigure, img, label):
  start = time.time()
  Y, X = trace_object_boundaries(label, conn=4, trace_all=True,
                                   simplify_colinear_spurs=True,
                                   eps_colinear_area=0.01)
  X, Y = zip(*[merge_colinear(x,y) for x,y in zip(X,Y)])
  print('Tracing time elapsed: {}'.format(time.time()-start))


  #offset by tile boundaries

  #get superpixel centroids
  centroids = regionprops_table(label, properties=['centroid'])
  cY = centroids['centroid-0']
  cX = centroids['centroid-1']
  
  fig, ax = plt.subplots(num=countFigure)
  ax.imshow(img)
  fig.gca().set_axis_off()
  fig.tight_layout()
  ax.scatter(cX, cY)
  for x,y in zip(X, Y):
    ax.plot(x, y)

In [ ]:
img = img_as_float(astronaut()[::2, ::2])
img = np.tile(img, (8,8,1))

#skimage
start = time.time()
label1 = slic(img, n_segments=int(img.shape[0]*img.shape[1]/4096.0), compactness=1, sigma=1, start_label=1)
print('skimage SLIC time elapsed: %s ' % format(time.time()-start))

#Simple ITK
start = time.time()
imgITK = sitk.GetImageFromArray(img, isVector=True)
labelITK = sitk.SLIC(imgITK, spatialProximityWeight=1.0, superGridSize=(64,64))
label2 = sitk.GetArrayFromImage(labelITK)
print('SITK SLIC time elapsed: %s ' % format(time.time()-start))

#fast_slic: algorithm is suspect
start = time.time()
slic = Slic(num_components=int(img.shape[0]*img.shape[1]/(64.0*64.0)), compactness=1)
imgFastSLIC = img.astype(np.uint8)
label3 = slic.iterate(imgFastSLIC)
print('fast_slic time elapsed: %s ' % format(time.time()-start))

In [ ]:
countFigure = 0
fig, ax = plt.subplots(num=countFigure)
countFigure = countFigure + 1
ax.imshow(img)

plotit(countFigure, img, label1)
countFigure = countFigure + 1
plotit(countFigure, img, label2)
countFigure = countFigure + 1
plotit(countFigure, img, label3)
countFigure = countFigure + 1